# Compute distortion matrix based on a checkerboard video

Code based on https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

In [ ]:
import os
from tqdm import tqdm
import cv2 as cv
import numpy as np

from notebooks.defisheye import DeFish

In [ ]:
# checkerboard dimensions (number of inner corners: (rows, columns) )
checkerboard_dims = (6, 8)

# path to extracted frames
frames_path = "../datasets/oor/checkerboard/frames"
# path to save detected checkerboards
detections_path = "../datasets/oor/checkerboard/detections"
# path to save undistorted images
undistorted_path = "../datasets/oor/checkerboard/undistorted"

# show checkerboard detections
show_checkerboards = False

## [Optional] Step 1: extract frames from video

Extract a chosen number of random frames from the video.

In [ ]:
# input video
checkerboard_video_path = "../datasets/oor/checkerboard/0-0-D26M03Y2024-H15M02S41.mp4"

# number of frames to use
n_frames = 30

os.makedirs(frames_path, exist_ok=True)

cap = cv.VideoCapture(checkerboard_video_path)
total_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))

# select and save random frames
frames = sorted(np.random.permutation(total_frames)[:n_frames])
for frame in tqdm(frames):
    cap.set(cv.CAP_PROP_POS_FRAMES, frame)
    res, img = cap.read()
    if res:
        out_path = os.path.join(frames_path, f"frame_{frame}_raw.jpg")
        cv.imwrite(out_path, img)
    else:
        print(f"Could not extract frame {frame} (out of {total_frames}), skipping.")

cap.release()

print(f"Extracted frames saved in {frames_path}. Please verify.")

## Step 2: detect checkerboard in frames

First, manually inspect frames to check if the checkerboard is fully visible in the frame and not blurry. Remove frames that do not meet these criteria.

In [ ]:
# termination criteria for checkerboard corner refinement
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((np.prod(checkerboard_dims), 3), np.float32)
objp[:,:2] = np.mgrid[0:checkerboard_dims[0],0:checkerboard_dims[1]].T.reshape(-1,2)
 
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

os.makedirs(detections_path, exist_ok=True)

frames = [file for file in os.listdir(frames_path) if file.endswith(".jpg")]
 
for frame in tqdm(frames):
    img = cv.imread(os.path.join(frames_path, frame))
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
 
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, checkerboard_dims, None)
 
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
 
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        # Draw the corners
        cv.drawChessboardCorners(img, checkerboard_dims, corners2, ret)
 
        if show_checkerboards:
            cv.imshow('img', img)
            cv.waitKey(500)
        
        frame_name, ext = os.path.splitext(frame)
        out_path = os.path.join(detections_path, f"{frame_name}_det{ext}")
        cv.imwrite(out_path, img)
    else:
        print(f"No checkerboard found in {frame}, skipped.")

cv.destroyAllWindows()

print(f"Detections saved in {detections_path}. Please verify.")

Manually verify that all detected checkerboards are precise. If certain frames do not result in precise detections, remove those form the frames folder and run **Step 2** again.

## Step 3: Compute the distortion correction parameters

This step uses the output of **Step 2**.

In [ ]:
# Compute calibration parameters
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Compute calibration error. Lower is better, in the order of 0.1 or less is acceptable.
total_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    total_error += error

print(f"Mean error: {total_error/len(objpoints):.3f}")

print("\nUse the following distortion correction parameters:")
print(f"\n- Camera matrix\n{mtx}")
print(f"\n- Distortion coefficients\n{dist}")

In [ ]:
# Optional: extract parameters needed for FFMPEG
n_decimals = 3
(w, h) = gray.shape[::-1]
ffmpeg_params = dict()
ffmpeg_params["cx"] = np.round(mtx[0,2] / w, decimals=n_decimals)
ffmpeg_params["cy"] = np.round(mtx[1,2] / h, decimals=n_decimals)
ffmpeg_params["k1"] = np.round(dist[0,0], decimals=n_decimals)
ffmpeg_params["k2"] = np.round(dist[0,1], decimals=n_decimals)

print(ffmpeg_params)

print(f"FFMPEG option: -vf lenscorrection=cx={ffmpeg_params['cx']}:cy={ffmpeg_params['cy']}:k1={ffmpeg_params['k1']}:k2={ffmpeg_params['k2']}:i=bilinear")

## Step 4: Undistort frames to verify the result

In [ ]:
distortion_params = {
    "camera_matrix": mtx,
    "distortion_params": dist,
    "input_image_size": gray.shape[::-1],
}

os.makedirs(undistorted_path, exist_ok=True)

frames = [file for file in os.listdir(frames_path) if file.endswith(".jpg")]

fish = DeFish(params=distortion_params)

for frame in tqdm(frames):
    image = cv.imread(os.path.join(frames_path, frame))
    image = fish.defisheye(image=image)
    cv.imwrite(filename=os.path.join(undistorted_path, frame), img=image)

print(f"Undistorted images saved in {undistorted_path}. Please verify.")